<img src="introduction.PNG">

# Introduction

Envision a use case involving a marketing team for a movie studio that wants to do 
targeted advertising each week before the weekend at the box-office. In order to do so, they must identify movies
that they believe will underperform in the upcoming weekend. The team therefore needs revenue predictions 
each week for all the movies at the box-office for the upcoming weekend. Before the weekend begins, they will 
use these predictions in order to formulate their targeted marketing campaigns. Here, we focus on the movie revenue 
predictions.

In this notebook, we create the master dataset to be used for the training and testing of a machine learning model that can 
predict weekend box office revenue. 

Here, a number of features are pulled in as part of the dataset to support this effort. The dataset is 
saved as a csv file so that it can be used by a machine learning framework for analysis (Azure ML Studio).

The resulting dataset is a matrix that we shall refer to as $\boldsymbol{X}$. Each row contains information about 
a single movie on a single weekend. The label vector, $\boldsymbol{y}$, contains the actual gross revenue 
recorded for the movie in the associated row of $\boldsymbol{X}$.

From this dataset, in the next notebook, we will create our feature correlations, $\rho_{i,\boldsymbol{y}}$, 
and our machine learning model, $\psi$.

Finally, as part of the data setup in this workbook, we create a dataset consisting of movies at the box-office for 
a single weekend (the upcoming weekend) to which we eventually apply the model, $\psi$, in a subsequent notebook. In this case,
$\psi$ is encapsulated in a deployed web-service created by Azure ML Studio.

General Azure ML Studio info:
-  https://docs.microsoft.com/en-us/azure/machine-learning/studio/what-is-ml-studio


# Data Source

http://www.the-numbers.com

# Import Libraries 

In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd
from IPython.core.debugger import Tracer


# Pull in top 5 actors in a movie

The gross revenue of a movie in a given weekend may depend on the actors that are in the movie.

In [3]:
# Pull the actors for a movie
def pull_actors(cast_and_crew_link):

    url = "http://www.the-numbers.com" + cast_and_crew_link;
    response = requests.get(url);
    page = response.text;
    soup = BeautifulSoup(page, "lxml");
    rows = soup.find_all(id="cast")[0:1]; #slice notation [0:1]
    actors = []
    for row in rows:
        cols = row.find_all('a')
        cols = [ele.text.strip() for ele in cols]
        actors.append(cols)

    ret = []
    
    if (len(actors) > 0):
        count = 0;
        for i in range(0, len(actors[0])):
            if (count >= 5):
                break

            ret.append(actors[0][i])
            count += 1;
        
    extra = 5 - len(ret)
    for i in range(0, extra):
        ret.append('None')
        
    return ret
    

In [4]:
actors = pull_actors("/movie/Mr-Magoo#tab=cast-and-crew")
print(actors)

['Leslie Nielsen', 'Kelly Lynch', 'Malcolm McDowell', 'Nick Chinlund', 'Stephen Tobolowsky']


# Pull in additional summary info - runtime, rating, genre, release type

Here, additional features are pulled that might provide additional predictive power for the gross revenue of a movie.

In [5]:
# Pulls additional summary info from
# for a movie such as run time, rating, and genre
def pull_summary(summary_link):

    url = "http://www.the-numbers.com" + summary_link;
    #url = "http://www.the-numbers.com/movie/Mr-Magoo#tab=summary"
    #print(url)
    response = requests.get(url);
    page = response.text;
    soup = BeautifulSoup(page, "lxml");
    tables = soup.find_all('table');
    sum_table = tables[3]
    tds = sum_table.find_all('td');
    ret = []
    
    count = 0;
    runtime = "90"
    rating = "None"
    genre = "None"
    release = "None"
    for td in tds:
        text = td.text
        
        # Run Time of Movie
        # Stripped to just be a number
        if (text == "Running Time:"):
            value = tds[count + 1].text;
            value = value.replace(' minutes', '')
            runtime = value 
            
        # Rating of the movie PG-13 / R etc.
        if ('MPAA' in text):
            value = tds[count + 1];
            a = value.find_all('a')
            rating = a[0].text
            
        # Genre of the movie
        if ('Genre' in text):
            value = tds[count + 1];
            genre = value.text
           
        # Wide vs Limtited domestic release
        if ('Domestic Releases' in text):
            value = tds[count + 1];
            val_str = str(value);
            start = val_str.index('(');
            stop = val_str.index(')');
            release = val_str[start+1:stop];
        
        count += 1;
    
    ret.append(runtime)
    ret.append(rating)
    ret.append(genre)
    ret.append(release)
    return ret

In [6]:
ret = pull_summary("/movie/Mr-Magoo#tab=summary")
print(ret)

['87', 'PG', 'Comedy', 'Wide']


# Create Master Dataset

Download weekend box office data from the-numbers.com for a given date range. This data will be used for the training and 
statistical validation of the machine learning models. Note that the download process is a bit slow and also that this function
is not very robust. If something fails in the middle of execution, the entire dataset is lost. This can be improved in a lot 
of places, but for now it serves its purpose.

Note also that predictions are only being made for movies that already exist at the box-office. In other words, opening weekends
for a movie are not being predicted.

The resulting dataset is matrix $\boldsymbol{X}$ with the 'gross' column as label vector $\boldsymbol{y}$.

**Output - Training and Validation:**
-  box_office_training_validation_dataset.csv
-  Note: Before uploading the csv file to Azure Studio make sure that all numerical columns are converted as such. For
    example the gross column contains money and therefore needs to be converted to numerical format.

**Associated Azure ML Studio Experiments (referenced in following notebook):**
-  https://gallery.cortanaintelligence.com/Experiment/Feature-Correlations-Weekend-Box-Office-Movie-Revenue
-  https://gallery.cortanaintelligence.com/Experiment/Forecasting-Weekend-Box-Office-Movie-Revenue-5

In [ ]:
#Tracer()() #this one triggers the debugger

# Download weekend box-office numbers for the follow date range.
# Note that the box-office revenue numbers for a week start on the friday of that week.
# Dates that aren't a friday, where the numbers are recorded, will be skipped.
#datelist = pd.date_range('2017-09-01', '2017-09-15').tolist()
datelist = pd.date_range('2010-01-01', '2017-11-14').tolist()
#datelist = pd.date_range('2011-01-01', '2011-03-01').tolist()

# Used for calculating some additional features
all_week_movie_gross_map = dict();
all_week_movie_theaters_map = dict();

movieData = []
for date in datelist:
    year = date.strftime('%Y')
    month = date.strftime('%m')
    day = date.strftime('%d')
    
    # Weekend box office
    url = "http://www.the-numbers.com/box-office-chart/weekend/"+year+"/"+month+"/"+day
    
    # Try twice in case it times out
    try:
        response = requests.get(url);
    except:
        response = requests.get(url);
        
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    rows = soup.find_all(id="page_filling_chart")[1].find_all('tr')[1:]
    
    # Iterate all movies for a given weekend
    x = 0;
    for row in rows:
        if x > 14: # Only top 15 movies per weekend
            break
                        
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]      
        
        # Last weeks gross map
        movie = cols[2];
        if movie not in all_week_movie_gross_map:
            all_week_movie_gross_map[movie] = ['0']
            all_week_movie_gross_map[movie].append(cols[4])
        else:
            vals = all_week_movie_gross_map[movie]
            vals.append(cols[4])
        
        # Last weeks theaters map
        if movie not in all_week_movie_theaters_map:
            all_week_movie_theaters_map[movie] = ['0']
            all_week_movie_theaters_map[movie].append(cols[6])
        else:
            vals = all_week_movie_theaters_map[movie]
            vals.append(cols[6])
        
        cols.insert(len(cols),date.strftime('%Y-%m-%d'))
        cols.insert(len(cols), year)
        cols.insert(len(cols), month)
        cols.insert(len(cols), day)
        
        # Get the cast and crew
        links = row.find_all('a')
        movie_link = links[0].get('href') # should be first one
        cast_and_crew_link = movie_link.replace('box-office', 'cast-and-crew') # need the right tab
        summary_link = movie_link.replace('box-office', 'summary') # need the right tab
        actors = pull_actors(cast_and_crew_link)
        summary_elements = pull_summary(summary_link)
        cols.extend(actors[0:5])
        cols.extend(summary_elements)
        
        # add previous week gross
        gross_vals = all_week_movie_gross_map[movie]
        gross_val = gross_vals[len(gross_vals)-2]
        cols.append(gross_val)
        
        # add previous week theaters
        theater_vals = all_week_movie_theaters_map[movie]
        theater_val = theater_vals[len(theater_vals)-2]
        cols.append(theater_val)
        
        # only movies that have been out for at least a week
        # no new releases
        if ((cols[1] != 'new') & (gross_val != '0')) :
            movieData.append(cols)
            x = x + 1
        
   # break
    
    # Keep track of how far we are because it takes forever
    if (len(rows) > 0) :
        print(date)

# Write the dataset to file
# This file can be pulled into a machine learning framework for analysis
#print(movieData)
df = pd.DataFrame(movieData)
allcols = ['rank', 'something', 'movie', 'distributor', 'gross', 'change']
allcols.extend(['theatres', 'per_theatre', 'total_gross','days','date'])
allcols.extend(['year', 'month', 'day',])
allcols.extend(['actor1','actor2','actor3','actor4','actor5',])
allcols.extend(['runtime','rating','genre','release_type','previous_week_gross','previous_week_theaters'])
df.columns = allcols
#df.to_csv('box_office_training_validation_dataset.csv')


# Create next week dataset

This chunk of code creates the dataset to be fed to a model to predict the revenue of the box office for a given weekend.
It assumes that the dates provided in the date range represent the weekend that has just passed and predictions are to 
be made for the upcoming weekend. Example - I want to predict the box office revenue for each movie for the weekend 
starting on Friday 11/17 - Set date range 2017-11-07 - 2017-11-14 so that data for the previous weekend is captured. 

**Output - Weekly Predictions:**
-  box_office_next_week_dataset.csv

For each row, $\boldsymbol{x_i}$, of the dataset, $\boldsymbol{X}$, we will eventually call our prediction function, 
$\psi(\boldsymbol{x_i})$ in a subsequent notebook.

**Associated Azure ML Studio Experiments (referenced in following notebook):**
-  https://gallery.cortanaintelligence.com/Experiment/Forecasting-Weekend-Box-Office-Movie-Revenue-Predictive-Exp

In [9]:
#Tracer()() #this one triggers the debugger

# Pulls data from the friday of 2017-11-10 to predict the revenue
# of movies for the week of 11-17-2017.
datelist = pd.date_range('2017-11-07', '2017-11-14').tolist()

movieData = []
for date in datelist:
    print(date)
    year = date.strftime('%Y')
    month = date.strftime('%m')
    day = date.strftime('%d')
    
    # Weekend box office
    url = "http://www.the-numbers.com/box-office-chart/weekend/"+year+"/"+month+"/"+day
    
    # Try twice in case it times out
    try:
        response = requests.get(url);
    except:
        response = requests.get(url);
        
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    rows = soup.find_all(id="page_filling_chart")[1].find_all('tr')[1:]
    
    # Iterate all movies for a given weekend
    x = 0;
    for row in rows:
        if x > 20: # Only top 15 movies per weekend
            break
                        
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]      
        
        # Last weeks gross map
        movie = cols[2];
        
        cols.insert(len(cols),date.strftime('%Y-%m-%d'))
        cols.insert(len(cols), year)
        cols.insert(len(cols), month)
        cols.insert(len(cols), day)
        
        # Get the cast and crew
        links = row.find_all('a')
        movie_link = links[0].get('href') # should be first one
        cast_and_crew_link = movie_link.replace('box-office', 'cast-and-crew') # need the right tab
        summary_link = movie_link.replace('box-office', 'summary') # need the right tab
        actors = pull_actors(cast_and_crew_link)
        summary_elements = pull_summary(summary_link)
        cols.extend(actors[0:5])
        cols.extend(summary_elements)
        
        # add previous week gross
        # in this case it's the same as this weeks gross
        # since we are predicting for next week
        gross_val = cols[4]
        cols.append(gross_val)
        
        # add previous week theaters
        # in this case it's the same as this weeks theaters
        # since we are predicting for next week
        theater_val = cols[6]
        cols.append(theater_val)
        
        # record the week
        movieData.append(cols)
        x = x + 1
        
   # break
    
    # Keep track of how far we are because it takes forever
    if (len(rows) > 0) :
        print(date)

# Write the dataset to file so that it can be fed to the deployed model web service
df = pd.DataFrame(movieData)
allcols = ['rank', 'something', 'movie', 'distributor', 'gross', 'change']
allcols.extend(['theatres', 'per_theatre', 'total_gross','days','date'])
allcols.extend(['year', 'month', 'day',])
allcols.extend(['actor1','actor2','actor3','actor4','actor5',])
allcols.extend(['runtime','rating','genre','release_type','previous_week_gross','previous_week_theaters'])
df.columns = allcols
df.to_csv('box_office_next_week_dataset.csv')

2017-11-07 00:00:00
2017-11-08 00:00:00
2017-11-09 00:00:00
2017-11-10 00:00:00
2017-11-10 00:00:00
2017-11-11 00:00:00
2017-11-12 00:00:00
2017-11-13 00:00:00
2017-11-14 00:00:00
